### Improving the code:

In [6]:
import boto3

import pandas as pd

from botocore import UNSIGNED

from botocore.config import Config

from io import StringIO, BytesIO

from datetime import datetime, timedelta

#### First, we want to add all possible parameters to the top to make the code more readable:

In [7]:
arg_date = '2022-02-13'

source_format = '%Y-%m-%d'

source_bucket = 'deutsche-boerse-xetra-pds'

target_bucket = 'xetra-bucket-123'

columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']

key = 'xetra_daily_report_' + datetime.today().strftime('%Y%m%d_%H%M%S') + '.parquet'

In [8]:
arg_date_dt = datetime.strptime(arg_date, source_format).date() - timedelta(days=1)

In [9]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

bucket = s3.Bucket(source_bucket)

objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], source_format).date() >= arg_date_dt]

#### Here we take everything that was inside the for loop below and turn it into a function, which allows our code to look much cleaner:

In [ ]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter =',')
    return df

df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True, axis=1)

In [ ]:
df_all

In [ ]:
df_all = df_all.loc[:, columns]

In [ ]:
df_all.dropna(inplace=True)

### Get opening price per ISIN and day:

In [ ]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

### Get closing price per ISIN and day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

### Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(
    opening_price_eur=('opening_price', 'min'),
    closing_price_eur=('closing_price', 'min'),
    minimum_price_eur=('MinPrice', 'min'),
    maximum_price_eur=('MaxPrice', 'max'),
    daily_traded_volume=('TradedVolume', 'sum')
)

### Percent Change Prev Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all['change_prev_closing_%'] = (
    (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
)

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all = df_all[df_all.Date >= arg_date]

### Write to S3:

In [ ]:
out_buffer = BytesIO()

df_all.to_parquet(out_buffer, index=False)

bucket_target = s3.Bucket(target_bucket)

bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

### Reading the uploaded file:

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20220212_000301.parquet').get().get('Body').read()

data = BytesIO(prq_obj)

df_report = pd.read_parquet(data)

In [ ]:
df_report